In [ ]:
# Cell 2: Quick Dataset Check
import yaml

with open('data.yaml', 'r') as f:
    data_cfg = yaml.safe_load(f)

print("📊 Dataset Configuration:")
print(f"Classes: {data_cfg['nc']}")
print(f"Class Names: {data_cfg['names']}")
print(f"Train path: {data_cfg['train']}")
print(f"Val path: {data_cfg['val']}")

In [ ]:
# Cell 3: Training (Single Cell)
from ultralytics import YOLO

# Load model and train
model = YOLO('yolov8n.pt')
# print(model.info)
results = model.train(
    data='data.yaml',
    epochs=4,
    imgsz=640,
    batch=8,
    device=0,
    name='yolov8n_grenade_mines',
    patience=15,
    save=True
)

print("🎉 Training completed!")

In [ ]:
# Cell 4: Quick Validation
model = YOLO('runs/detect/yolov8n_grenade_mines/weights/best.pt')
metrics = model.val()
print(f"📈 mAP50-95: {metrics.box.map:.4f}")

In [ ]:
# Cell 5: Export for Raspberry Pi
model.export(format='onnx')
print("✅ Model exported to ONNX format")

In [ ]:
import glob
import random
import matplotlib.pyplot as plt
import os

# Make predictions
results = model.predict('test/images/', save=True, conf=0.5)
print(f"🔍 Predictions saved in: {results[0].save_dir}")

In [ ]:
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

# Quantize model from FP32 to INT8
def quantize_model():
    input_model = 'best.onnx'
    output_model = 'best_qnt.onnx'
    
    quantize_dynamic(
        input_model,
        output_model,
        weight_type=QuantType.QUInt8
    )
    
    print(f"✅ Model quantized: {output_model}")
    print("📊 Size reduced for faster Raspberry Pi inference")

In [9]:
if __name__ == "__main__":
    quantize_model()

ValidationError: Unable to open proto file: best.onnx. Please check if it is a valid proto. 